In [1]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()  # <1>
    except AttributeError:  # no .replace or .split
        pass  # assume it's already a sequence of identifiers
    field_names = tuple(field_names)  # <2>

    def __init__(self, *args, **kwargs):  # <3>
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self):  # <4>
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):  # <5>
        values = ', '.join('{}={!r}'.format(*i) for i
                           in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__ = field_names,  # <6>
                     __init__  = __init__,
                     __iter__  = __iter__,
                     __repr__  = __repr__)

    return type(cls_name, (object,), cls_attrs)  # <7>

#### record_factory: create simple classes just for holding data fields

In [2]:
>>> Dog = record_factory('Dog', 'name weight owner')  # <1>
>>> rex = Dog('Rex', 30, 'Bob')
>>> rex  # <2>

Dog(name='Rex', weight=30, owner='Bob')

In [3]:
>>> name, weight, _ = rex  # <3>
>>> name, weight

('Rex', 30)

In [4]:
>>> "{2}'s dog weighs {1}kg".format(*rex)  # <4>

"Bob's dog weighs 30kg"

In [5]:
>>> rex.weight = 32  # <5>
>>> rex

Dog(name='Rex', weight=32, owner='Bob')

In [6]:
>>> Dog.__mro__  # <6>

(__main__.Dog, object)

#### The factory also accepts a list or tuple of identifiers:

In [7]:
>>> Dog = record_factory('Dog', ['name', 'weight', 'owner'])
>>> Dog.__slots__

('name', 'weight', 'owner')

In [8]:
Dog.__dict__

mappingproxy({'__slots__': ('name', 'weight', 'owner'),
              '__init__': <function __main__.record_factory.<locals>.__init__(self, *args, **kwargs)>,
              '__iter__': <function __main__.record_factory.<locals>.__iter__(self)>,
              '__repr__': <function __main__.record_factory.<locals>.__repr__(self)>,
              '__module__': '__main__',
              'name': <member 'name' of 'Dog' objects>,
              'owner': <member 'owner' of 'Dog' objects>,
              'weight': <member 'weight' of 'Dog' objects>,
              '__doc__': None})